In [143]:
import plotly.graph_objects as go
import pandas as pd

import matplotlib.pyplot as plt
import networkx as nx

# reading input log file
# import ruamel.yaml
import yaml
import re
import ast

import math
import sys
import traceback
from csv import excel
from collections import Counter
from scipy import stats

# test_name = "1p9f9s_run"
test_name = "seq6f3s"

stat_path=f"../example_stat/{test_name}"
iamge_path=f"{stat_path}/images"


In [144]:
import os
import re

def search_files_with_name(directory, pattern=""):
    file_list = []
    for root, dirs, files in os.walk(directory):
        print(root, dirs, files)
        for file in files:
            if re.search(pattern, file):
                file_list.append(os.path.join(root, file))
                #print(os.path.join(root, file))
    return file_list

vfd_files = search_files_with_name(stat_path, "vfd")
# vfd_files = vfd_files[0:1]
print(vfd_files)

../example_stat/seq6f3s [] ['apriori_seq6f3s.yaml', '32510_vfd-data-stat-dl.yaml', 'seq6f3s-task_to_file.yaml', '32510_vol-data-stat-dl.yaml']
['../example_stat/seq6f3s/32510_vfd-data-stat-dl.yaml']


In [145]:

def load_vfd_yaml(vfd_files):
    # loag into {file_name:yaml_data} format
    ret_dict = {}
    tmp_dict = {}
    for f in vfd_files:
        with open(f, "r") as stream:
            print(f"loading {f}")
            try:
                tmp_dict = yaml.safe_load(stream)
                ret_dict[f] = tmp_dict
                # print(tmp_dict)
            except yaml.YAMLError as exc:
                print(exc)
                print("Error loading yaml file")
                exit(1)
    return ret_dict

vfd_file_dict = load_vfd_yaml(vfd_files)
print(vfd_file_dict.keys())


loading ../example_stat/seq6f3s/32510_vfd-data-stat-dl.yaml
dict_keys(['../example_stat/seq6f3s/32510_vfd-data-stat-dl.yaml'])


In [146]:
# task_order_list = [ "run_idfeature", "run_tracksingle", "run_gettracks", "run_trackstats", "run_identifymcs", "run_matchpf", "run_mcsstats", "run_robustmcs", "run_mapfeature", "run_speed"]


def extract_pid_from_file_path(file_path):
    # Define the regular expression pattern to find numbers in the file path
    pattern = r'\d+'
    # Use the re.findall function to find all occurrences of the pattern in the file path
    numbers = re.findall(pattern, file_path)
    # Since there might be multiple numbers in the file path, you can extract the last one
    if numbers:
        desired_number = int(numbers[-1])
        print("task_pid:", desired_number)
    else:
        print("No task_pid found in the file path.")
    return desired_number

def get_op_blob_range(h5fd_stat):
    # Format of io_stat
    '''
    read_bytes: 0
        read_cnt: 0
        read_ranges: {1:[0, 0]}
        write_bytes: 7746756
        write_cnt: 52
        write_ranges:{2:[0, 0]}
    '''
    read_ranges = h5fd_stat['read_ranges']
    write_ranges = h5fd_stat['write_ranges']
    
    if read_ranges:
        start_op_idx = min(read_ranges.keys())
        end_op_idx = max(read_ranges.keys())
        start_blob = min(elem[0] for elem in list(read_ranges.values())) #min(read_ranges.values())
        end_blob = max(elem[1] for elem in list(read_ranges.values()))
    if write_ranges:
        start_op_idx = min(write_ranges.keys())
        end_op_idx = max(write_ranges.keys())
        start_blob = min(elem[0] for elem in list(write_ranges.values())) #min(write_ranges.values())
        end_blob = max(elem[1] for elem in list(write_ranges.values()))
    
    result = {
        'start_op_idx': start_op_idx,
        'end_op_idx': end_op_idx,
        'start_blob': start_blob,
        'end_blob': end_blob
    }
    
    return result

# TODO: need different get_min_max_op_idx
def get_min_max_op_idx(io_stat):
    # assumes we want the I/O operations and blobs for per file
    min_op_idx = -1
    max_op_idx = -1
    min_blob = -1
    max_blob = -1
    for h5fd_type, h5fd_stat in io_stat.items():
        res_stat = get_op_blob_range(h5fd_stat)
        
        if min_op_idx == -1:
            min_op_idx = res_stat['start_op_idx']
        elif res_stat['start_op_idx'] < min_op_idx:
            min_op_idx = res_stat['start_op_idx']
        
        if max_op_idx == -1:
            max_op_idx = res_stat['end_op_idx']
        elif res_stat['end_op_idx'] > max_op_idx:
            max_op_idx = res_stat['end_op_idx']
        
        if min_blob == -1:
            min_blob = res_stat['start_blob']
        elif res_stat['start_blob'] < min_blob:
            min_blob = res_stat['start_blob']
        
        if max_blob == -1:
            max_blob = res_stat['end_blob']
        elif res_stat['end_blob'] > max_blob:
            max_blob = res_stat['end_blob']
    
    result = {
        'min_op_idx': min_op_idx,
        'max_op_idx': max_op_idx,
        'min_blob': min_blob,
        'max_blob': max_blob
    }
    
    # print(f"h5fd_type: {h5fd_type}")
    # print(f"res_stat: {result}")
    
    return result


# Converting to Task-File dictionary
def schema_tp_task_file_dic(vfd_file_dict):
    task_file_dict = {}
    # Format of task_file_dict
    '''
    run_idfeature:
        order: 0
        task_pid: 75190
        task_op_range: [0, 7208]
        total_op_range: [0, 7208]
        input:
            - file_name: "wrfout_rainrate_tb_zh_mh_2015-05-06_04:00:00.nc"
                file_op_range: [55, 71]
        output:
            - file_name: "cloudid_20150506_040000.nc"
                ile_op_range: [0, 800]
    '''
    for file_path, file_data in vfd_file_dict.items():

        task_pid = extract_pid_from_file_path(file_path)
        
        for li in file_data:
            # print(li.keys())
            if 'file' in list(li.keys())[0]:
                key = list(li.keys())[0]
                # print(li[key])
                task_name = li[key]['task_name']
                file_name = li[key]['file_name']

                if task_name not in task_file_dict.keys():
                    # 1. Initialize task_file_dict
                    task_file_dict[task_name] = {}
                    task_file_dict[task_name]['order'] = 0 # placeholder
                    task_file_dict[task_name]['task_pid'] = task_pid
                    task_file_dict[task_name]['io_cnt'] = 0 # initial value
                    # task_file_dict[task_name]['total_op_range'] = []
                    task_file_dict[task_name]['input'] = {}
                    task_file_dict[task_name]['output'] = {}
                    
                # 1. Update task op count
                read_cnt = li[key]['file_read_cnt']
                write_cnt = li[key]['file_write_cnt']
                task_file_dict[task_name]['io_cnt'] += (read_cnt + write_cnt)
                
                # 2. Get blob and operation info
                data_stat = li[key]['data']#['H5FD_MEM_DRAW']
                meta_stats = li[key]['metadata']
                data_stat.update(meta_stats)
                io_stat = get_min_max_op_idx(data_stat)
                
                # 2. Add file to intput/output list
                file_type = li[key]['file_type']
                if file_type == 'input':
                    # task_file_dict[task_name]['input'].append(file_name)
                    task_file_dict[task_name]['input'][file_name] = io_stat
                elif file_type == 'output':
                    # task_file_dict[task_name]['output'].append(file_name)
                    task_file_dict[task_name]['output'][file_name] = io_stat
                # TODO: what if read_write file occur?
                

                # print(f"{file_name}: {io_stat}")
                # data_stat_range_list = list(data_stat['write_ranges'].values())
                
    return task_file_dict

def sort_task_in_order(task_file_dict, task_order_list):
    op_offset = 0
    for i, task_name in enumerate(task_order_list):
        # if task_name in task_file_dict.keys():
        task_file_dict[task_name]['order'] = i
        io_cnt = task_file_dict[task_name]['io_cnt']
        # task_file_dict[task_name]['total_op_range'] = f"[{op_offset},{io_cnt}]"
        task_file_dict[task_name]['total_op_range'] = {}
        
        task_file_dict[task_name]['total_op_range']['start'] = op_offset
        task_file_dict[task_name]['total_op_range']['end'] = op_offset + io_cnt

        op_offset += io_cnt
    return task_file_dict


In [147]:
task_file_dict = schema_tp_task_file_dic(vfd_file_dict)
task_order_list = [ "run_idfeature", "run_tracksingle", "run_gettracks"]
ordered_task_file_dict = sort_task_in_order(task_file_dict, task_order_list)

for task_name,data in task_file_dict.items():
    print(f"{task_name}:")
    print(f"    {data}")



task_pid: 32510
run_idfeature:
    {'order': 0, 'task_pid': 32510, 'io_cnt': 4824, 'input': {'wrfout_rainrate_tb_zh_mh_2015-05-06_00:00:00.nc': {'min_op_idx': 0, 'max_op_idx': 74, 'min_blob': 0, 'max_blob': 33}, 'wrfout_rainrate_tb_zh_mh_2015-05-06_01:00:00.nc': {'min_op_idx': 804, 'max_op_idx': 878, 'min_blob': 0, 'max_blob': 33}, 'wrfout_rainrate_tb_zh_mh_2015-05-06_02:00:00.nc': {'min_op_idx': 1608, 'max_op_idx': 1682, 'min_blob': 0, 'max_blob': 32}, 'wrfout_rainrate_tb_zh_mh_2015-05-06_03:00:00.nc': {'min_op_idx': 2412, 'max_op_idx': 2486, 'min_blob': 0, 'max_blob': 31}, 'wrfout_rainrate_tb_zh_mh_2015-05-06_04:00:00.nc': {'min_op_idx': 3216, 'max_op_idx': 3290, 'min_blob': 0, 'max_blob': 31}, 'wrfout_rainrate_tb_zh_mh_2015-05-06_05:00:00.nc': {'min_op_idx': 4020, 'max_op_idx': 4094, 'min_blob': 0, 'max_blob': 30}}, 'output': {'cloudid_20150506_000000.nc': {'min_op_idx': 75, 'max_op_idx': 803, 'min_blob': 0, 'max_blob': 14}, 'cloudid_20150506_010000.nc': {'min_op_idx': 879, 'max_op_

In [148]:
# Save the task to input/output file mapping
def save_task_file_dict(task_file_dict, test_name):
    tf_file_path = stat_path=f"../example_stat/{test_name}/{test_name}-task_to_file.yaml"
    
    # if os.path.exists(tf_file_path):
    #     # Remove the file
    #     os.remove(tf_file_path)
    
    with open(tf_file_path, 'w') as yaml_file:
        for task_name,data in task_file_dict.items():
            input_files = list(data['input'].keys())
            output_files = list(data['output'].keys())
            item_dict = {
                task_name: {
                    'order': data['order'],
                    'task_pid': data['task_pid'], # TODO: not need?
                    'io_cnt': data['io_cnt'], # TODO: not need?
                    'input': input_files,
                    'output': output_files
                }
            }
            yaml.dump(item_dict, yaml_file)

save_task_file_dict(task_file_dict, test_name)

In [149]:
# Convert dictionary to prefetcher format
# TODO: Need to convert to file_name based dictionary first

prefetch_file_path = stat_path=f"../example_stat/{test_name}/apriori_{test_name}.yaml"
file_op_dict = {}

# if os.path.exists(prefetch_file_path):
#     # Remove the file
#     os.remove(prefetch_file_path)

with open(prefetch_file_path, 'w') as file:
    file.write("0:\n")
    for task_name,data in task_file_dict.items():
        for input_file,file_stat in data['input'].items():
            op_range = f"[{file_stat['min_op_idx']},{file_stat['max_op_idx']}]"
            blob_range = f"[{file_stat['min_blob']},{file_stat['max_blob']}]"
            file.write(f"  - bucket: {input_file}\n")
            file.write(f"    prefetch:\n")
            file.write(f"    - op_count_range: {op_range}\n")
            file.write(f"      promote_blobs: {blob_range}\n")
        for output_file,file_stat in data['input'].items():
            op_range = f"[{file_stat['min_op_idx']},{file_stat['max_op_idx']}]"
            blob_range = f"[{file_stat['min_blob']},{file_stat['max_blob']}]"
            file.write(f"  - bucket: {input_file}\n")
            file.write(f"    prefetch:\n")
            file.write(f"    - op_count_range: {op_range}\n")
            file.write(f"      demote_blobs: {blob_range}\n")
            # print(input_file)
            # print(file_stat)
    
    

In [150]:
# op_range_shift_dict = {
#     "75190":0,
#     "5904":7208,
#     "77100":12689,
#     "168881":15641,
#     "98522":30542,
#     "170958":31599,
#     "11179":34399,
#     "157121":37801,
#     # "167838":91681,
# }



# task_pid_dict = {
#     "run_idfeature": {"task_pid":75190},
#     "run_tracksingle": {"task_pid":5904},
#     "run_gettracks": {"task_pid":77100},
#     "run_trackstats": {"task_pid":168881},
#     "run_identifymcs": {"task_pid":98522},
#     "run_matchpf": {"task_pid":170958},
#     "run_mcsstats": {"task_pid":11179},
#     "run_robustmcs": {"task_pid":157121},
#     # "run_mapfeature": {"task_pid":167838},
#     # "run_speed": {"task_pid":??},
# }



# prefetch_entry_list = []

# # get the I/O range
# for k,stat in vfd_dict.items():
#     # print(f"VFD Stat File: {k}")
#     task_max_io = 0
#     prev_file_max_io = 0
#     file_op_range_dict = {}
#     for li in stat:
#         file_key = list(li.keys())[0]
#         if 'file' in file_key:
#             # file_stat = list(li.values())[0]
#             file_stat = li[file_key]
#             file_name = file_stat['file_name']
#             file_max_io = 0
#             file_min_io = prev_file_max_io
#             file_max_blob = 0
#             prefetch_type = 0 # 0: no prefetch, 1: prefetch blobs
#             try:
#                 h5mDraw = file_stat['data']['H5FD_MEM_DRAW']
                
#                 if h5mDraw['read_cnt'] > 0:
#                     # print(f"blobs here: {[val[1] for val in h5mDraw['read_ranges'].values()]}")
#                     max_blob = max([val[1] for val in h5mDraw['read_ranges'].values()])
#                     max_io = max(h5mDraw['read_ranges'].keys())
#                     min_io = min(h5mDraw['read_ranges'].keys())
#                     if max_blob > file_max_blob: file_max_blob = max_blob
#                     if max_io > task_max_io: task_max_io = max_io
#                     if max_io > file_max_io: file_max_io = max_io
#                     if min_io < file_min_io: file_min_io = min_io
#                     prefetch_type = 1
                    
#                 elif h5mDraw['write_cnt'] > 0:
#                     # print(f"blobs here: {[val[1] for val in h5mDraw['write_ranges'].values()]}")
#                     max_blob = max([val[1] for val in h5mDraw['write_ranges'].values()])
#                     max_io = max(h5mDraw['write_ranges'].keys())
#                     min_io = min(h5mDraw['read_ranges'].keys())
#                     if max_blob > file_max_blob: file_max_blob = max_blob
#                     if max_io > task_max_io: task_max_io = max_io
#                     if max_io > file_max_io: file_max_io = max_io
#                     if min_io < file_min_io: file_min_io = min_io
#                     prefetch_type = 0
#                 else:
#                     pass
                
#                 for meta_type, meta_stats in file_stat['metadata'].items():
#                     # print(f"meta_type: {meta_type}")
#                     if file_stat['metadata'][meta_type]['read_cnt'] > 0:
#                         max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['read_ranges'].values()])
#                         max_io = max(file_stat['metadata'][meta_type]['read_ranges'].keys())
#                         min_io = min(file_stat['metadata'][meta_type]['read_ranges'].keys())
#                         if max_blob > file_max_blob: file_max_blob = max_blob
#                         if max_io > task_max_io: task_max_io = max_io
#                         if max_io > file_max_io: file_max_io = max_io
#                         if min_io < file_min_io: file_min_io = min_io
#                         prefetch_type = 1
                        
#                     elif file_stat['metadata'][meta_type]['write_cnt'] > 0:
#                         max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['write_ranges'].values()])
#                         max_io = max(file_stat['metadata'][meta_type]['write_ranges'].keys())
#                         min_io = min(file_stat['metadata'][meta_type]['write_ranges'].keys())
#                         if max_blob > file_max_blob: file_max_blob = max_blob
#                         if max_io > task_max_io: task_max_io = max_io
#                         if max_io > file_max_io: file_max_io = max_io
#                         if min_io < file_min_io: file_min_io = min_io
#                         prefetch_type = 0
#                     else:
#                         pass
                
#                 # if "cloudid_20150506_040000" in file_name:
#                 #     print(f"VFD Stat File: {k}")
#                 #     print(f"file_name: {file_name}")
#                 #     print(f"file_io_range: [{file_min_io},{file_max_io}]")
#                 if file_name in file_op_range_dict.keys():
#                     file_op_range_dict[file_name]['io_ranges'] .append((file_min_io, file_max_io))
#                     if file_max_blob != file_op_range_dict[file_name]['blob_reanges'][1]:
#                         file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
#                         file_op_range_dict[file_name]['prefetch_type'] = prefetch_type
                        
#                 else:
#                     file_op_range_dict[file_name] = {}
#                     file_op_range_dict[file_name]['io_ranges'] = [(file_min_io, file_max_io)]
#                     file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
#                     file_op_range_dict[file_name]['prefetch_type'] = prefetch_type
            
#                 prev_file_max_io = file_max_io
#                 if prefetch_type == 0:
#                     print(f"file_op_range_dict: {file_op_range_dict[file_name]}")

#             except:
#                 pass


#     file_name = k.split('/')[-1]
#     task_id = file_name.split('_')[0]
#     # print(f"{task_id} Max IO IDX: {task_max_io}")
#     range_shift = op_range_shift_dict[task_id]
#     page_size_factor = 1 # comparing to 1M, 512KB is 2 for 1M
    
#     for file_name, ranges_val in file_op_range_dict.items():
#         for range in ranges_val['io_ranges']:
#             pre_entry = {}
#             # pre_entry['op_count_range'] = f"[{page_size_factor*(range_shift+0)}, {page_size_factor*(range_shift+task_max_io)}]"
#             # pre_entry['op_count_range'] = [page_size_factor*(range_shift+range[0]), page_size_factor*(range_shift+range[1])]
#             pre_entry['op_count_range'] = [page_size_factor*(range_shift), page_size_factor*(range_shift)]


#             pre_entry['prefetch'] = []
#             bucket_entry = {}
#             bucket_entry['bucket'] = file_name
#             # bucket_entry['promote_blobs'] = f"[{page_size_factor*(ranges_val['blob_reanges'][0])}, {page_size_factor*(ranges_val['blob_reanges'][1])}]"
#             bucket_entry['blob_ranges'] = [page_size_factor*(ranges_val['blob_reanges'][0]), page_size_factor*(ranges_val['blob_reanges'][1])]
#             bucket_entry['prefetch_type'] = "promote_blobs" if ranges_val['prefetch_type'] == 1 else "demote_blobs"
            
#             pre_entry['prefetch'].append(bucket_entry)
#             prefetch_entry_list.append(pre_entry)
            

# with open('./apriori_p-1p9f-512KB.yml', 'w') as outfile:
#     outfile.writelines("0:\n")
#     sorted_entries = sorted(prefetch_entry_list, key=lambda x: x['op_count_range'][0])

#     for ent in sorted_entries:
#         # print(f"  - op_count_range: {ent['op_count_range']}")
#         # print(f"    prefetch:")
        
#         outfile.writelines(f"  - op_count_range: {ent['op_count_range']}\n")
#         outfile.writelines(f"    prefetch:\n")
#         for bucket in ent['prefetch']:
#             # print(f"      - bucket: \"{bucket['bucket']}\"")
#             # print(f"        promote_blobs: {bucket['promote_blobs']}")
#             # TODO: bucketname correction
#             prefix = bucket['bucket'].split('_')[0]
#             if prefix == "cloudid" or prefix == "track":
#                 prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_wrf_tbradar/tracking"
#             if prefix == "mcs" or prefix == "trackstats":
#                 prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_wrf_tbradar/stats"
#             if prefix == "wrfout" or prefix == "wrf":
#                 prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_input_data/wrf_tbradar"
#             bucket_name = prefix_path + "/" + bucket['bucket']
#             outfile.writelines(f"      - bucket: \"{bucket_name}\"\n")
#             outfile.writelines(f"        {bucket['prefetch_type']}: {bucket['blob_ranges']}\n")

    

In [151]:
op_range_shift_dict = {
    "75190":0,
    "5904":7208,
    "77100":12689,
    "168881":15641,
    "98522":30542,
    "170958":31599,
    "11179":34399,
    "157121":37801,
    # "167838":91681,
}

task_pid_dict = {
    "run_idfeature": {"task_pid":75190},
    "run_tracksingle": {"task_pid":5904},
    "run_gettracks": {"task_pid":77100},
    "run_trackstats": {"task_pid":168881},
    "run_identifymcs": {"task_pid":98522},
    "run_matchpf": {"task_pid":170958},
    "run_mcsstats": {"task_pid":11179},
    "run_robustmcs": {"task_pid":157121},
    # "run_mapfeature": {"task_pid":167838},
    # "run_speed": {"task_pid":??},
}



prefetch_entry_list = []

# get the I/O range
for k,stat in vfd_dict.items():
    # print(f"VFD Stat File: {k}")
    task_max_io = 0
    prev_file_max_io = 0
    file_op_range_dict = {}
    for li in stat:
        file_key = list(li.keys())[0]
        if 'file' in file_key:
            # file_stat = list(li.values())[0]
            file_stat = li[file_key]
            file_name = file_stat['file_name']
            file_max_io = 0
            file_min_io = prev_file_max_io
            file_max_blob = 0
            prefetch_type = 0 # 0: no prefetch, 1: prefetch blobs
            try:
                h5mDraw = file_stat['data']['H5FD_MEM_DRAW']
                
                if h5mDraw['read_cnt'] > 0:
                    # print(f"blobs here: {[val[1] for val in h5mDraw['read_ranges'].values()]}")
                    max_blob = max([val[1] for val in h5mDraw['read_ranges'].values()])
                    max_io = max(h5mDraw['read_ranges'].keys())
                    min_io = min(h5mDraw['read_ranges'].keys())
                    if max_blob > file_max_blob: file_max_blob = max_blob
                    if max_io > task_max_io: task_max_io = max_io
                    if max_io > file_max_io: file_max_io = max_io
                    if min_io < file_min_io: file_min_io = min_io
                    prefetch_type = 1
                    
                elif h5mDraw['write_cnt'] > 0:
                    # print(f"blobs here: {[val[1] for val in h5mDraw['write_ranges'].values()]}")
                    max_blob = max([val[1] for val in h5mDraw['write_ranges'].values()])
                    max_io = max(h5mDraw['write_ranges'].keys())
                    min_io = min(h5mDraw['read_ranges'].keys())
                    if max_blob > file_max_blob: file_max_blob = max_blob
                    if max_io > task_max_io: task_max_io = max_io
                    if max_io > file_max_io: file_max_io = max_io
                    if min_io < file_min_io: file_min_io = min_io
                    prefetch_type = 0
                else:
                    pass
                
                for meta_type, meta_stats in file_stat['metadata'].items():
                    # print(f"meta_type: {meta_type}")
                    if file_stat['metadata'][meta_type]['read_cnt'] > 0:
                        max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['read_ranges'].values()])
                        max_io = max(file_stat['metadata'][meta_type]['read_ranges'].keys())
                        min_io = min(file_stat['metadata'][meta_type]['read_ranges'].keys())
                        if max_blob > file_max_blob: file_max_blob = max_blob
                        if max_io > task_max_io: task_max_io = max_io
                        if max_io > file_max_io: file_max_io = max_io
                        if min_io < file_min_io: file_min_io = min_io
                        prefetch_type = 1
                        
                    elif file_stat['metadata'][meta_type]['write_cnt'] > 0:
                        max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['write_ranges'].values()])
                        max_io = max(file_stat['metadata'][meta_type]['write_ranges'].keys())
                        min_io = min(file_stat['metadata'][meta_type]['write_ranges'].keys())
                        if max_blob > file_max_blob: file_max_blob = max_blob
                        if max_io > task_max_io: task_max_io = max_io
                        if max_io > file_max_io: file_max_io = max_io
                        if min_io < file_min_io: file_min_io = min_io
                        prefetch_type = 0
                    else:
                        pass
                
                # if "cloudid_20150506_040000" in file_name:
                #     print(f"VFD Stat File: {k}")
                #     print(f"file_name: {file_name}")
                #     print(f"file_io_range: [{file_min_io},{file_max_io}]")
                if file_name in file_op_range_dict.keys():
                    file_op_range_dict[file_name]['io_ranges'] .append((file_min_io, file_max_io))
                    if file_max_blob != file_op_range_dict[file_name]['blob_reanges'][1]:
                        file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
                        file_op_range_dict[file_name]['prefetch_type'] = prefetch_type
                        
                else:
                    file_op_range_dict[file_name] = {}
                    file_op_range_dict[file_name]['io_ranges'] = [(file_min_io, file_max_io)]
                    file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
                    file_op_range_dict[file_name]['prefetch_type'] = prefetch_type
            
                prev_file_max_io = file_max_io
                if prefetch_type == 0:
                    print(f"file_op_range_dict: {file_op_range_dict[file_name]}")

            except:
                pass


    file_name = k.split('/')[-1]
    task_id = file_name.split('_')[0]
    # print(f"{task_id} Max IO IDX: {task_max_io}")
    range_shift = op_range_shift_dict[task_id]
    page_size_factor = 1 # comparing to 1M, 512KB is 2 for 1M
    
    for file_name, ranges_val in file_op_range_dict.items():
        for range in ranges_val['io_ranges']:
            pre_entry = {}
            # pre_entry['op_count_range'] = f"[{page_size_factor*(range_shift+0)}, {page_size_factor*(range_shift+task_max_io)}]"
            # pre_entry['op_count_range'] = [page_size_factor*(range_shift+range[0]), page_size_factor*(range_shift+range[1])]
            pre_entry['op_count_range'] = [page_size_factor*(range_shift), page_size_factor*(range_shift)]


            pre_entry['prefetch'] = []
            bucket_entry = {}
            bucket_entry['bucket'] = file_name
            # bucket_entry['promote_blobs'] = f"[{page_size_factor*(ranges_val['blob_reanges'][0])}, {page_size_factor*(ranges_val['blob_reanges'][1])}]"
            bucket_entry['blob_ranges'] = [page_size_factor*(page_size_factor*ranges_val['blob_reanges'][0]), page_size_factor*(page_size_factor*ranges_val['blob_reanges'][1])]
            bucket_entry['prefetch_type'] = "promote_blobs" if ranges_val['prefetch_type'] == 1 else "demote_blobs"
            
            pre_entry['prefetch'].append(bucket_entry)
            prefetch_entry_list.append(pre_entry)
            

with open('./apriori_p-1p9f-512KB.yml', 'w') as outfile:
    outfile.writelines("0:\n")
    sorted_entries = sorted(prefetch_entry_list, key=lambda x: x['op_count_range'][0])

    for ent in sorted_entries:
        # print(f"  - op_count_range: {ent['op_count_range']}")
        # print(f"    prefetch:")
        
        outfile.writelines(f"  - op_count_range: {ent['op_count_range']}\n")
        outfile.writelines(f"    prefetch:\n")
        for bucket in ent['prefetch']:
            # print(f"      - bucket: \"{bucket['bucket']}\"")
            # print(f"        promote_blobs: {bucket['promote_blobs']}")
            # TODO: bucketname correction
            prefix = bucket['bucket'].split('_')[0]
            if prefix == "cloudid" or prefix == "track":
                prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_wrf_tbradar/tracking"
            if prefix == "mcs" or prefix == "trackstats":
                prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_wrf_tbradar/stats"
            if prefix == "wrfout" or prefix == "wrf":
                prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_input_data/wrf_tbradar"
            bucket_name = prefix_path + "/" + bucket['bucket']
            outfile.writelines(f"      - bucket: \"{bucket_name}\"\n")
            outfile.writelines(f"        {bucket['prefetch_type']}: {bucket['blob_ranges']}\n")


KeyError: '32510'